<a href="https://colab.research.google.com/github/newning117/DATATHON_4/blob/main/%EA%B9%80%ED%95%98%EB%8A%98/%EB%A9%8B%EC%82%AC_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%86%A4_4%ED%8C%80_%EA%B9%80%ED%95%98%EB%8A%98_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**데이터 전처리**


In [ ]:
!pip install prophet

In [ ]:
!sed -i 's/np.float_/np.float64/g' /usr/local/lib/python3.*/dist-packages/prophet/forecaster.py
!sed -i 's/np.float_/np.float64/g' /usr/local/lib/python3.*/dist-packages/prophet/models.py


In [ ]:
from prophet import Prophet
print("✅ 개발 버전 Prophet (NumPy2.0 호환) 설치 완료")



✅ 개발 버전 Prophet (NumPy2.0 호환) 설치 완료


##**1. 데이터 경로설정 후 불러오기+형식확인**

In [ ]:
# 1. 라이브러리 설치 및 불러오기
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


#데이터 불러오기

path = "/content/drive/MyDrive/멋사_데이터톤/머신러닝/data/전처리_정제_2.csv"
df = pd.read_csv(path)

In [ ]:
# ② 예측할 대상 변수 선택

target_col = "신규고객_비중_cleaned"

# Prophet용 컬럼 준비
df["기준년월_dt"] = pd.to_datetime(df["기준년월_dt"])
df = df[["가맹점구분번호", "기준년월_dt", target_col]].rename(
    columns={"기준년월_dt": "ds", target_col: "y"}
)

In [ ]:
# ③ 전체 가맹점 리스트

store_list = df["가맹점구분번호"].unique()
print(f"총 {len(store_list)}개 가맹점 예측 진행 중...")


총 4172개 가맹점 예측 진행 중...


In [ ]:
# ④ 결과 저장용 리스트

results = []

In [ ]:
# ⑤ Prophet 반복 예측

for store_id in store_list:
    df_store = df[df["가맹점구분번호"] == store_id].sort_values("ds").reset_index(drop=True)

    # 데이터 충분한 가맹점만 예측 (최소 13개월)
    if len(df_store) < 13:
        continue

    # train/test 분리
    train = df_store[(df_store["ds"] >= "2023-11-01") & (df_store["ds"] <= "2024-11-01")]
    test  = df_store[df_store["ds"] == "2024-12-01"]

    # 테스트 데이터가 존재하지 않으면 skip
    if test.empty:
        continue

    # Prophet 모델 정의
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="additive"
    )

    # 모델 학습
    model.fit(train)

    # 미래 1개월 예측
    future = model.make_future_dataframe(periods=1, freq="MS")
    forecast = model.predict(future)

    # 실제값/예측값 비교
    y_true = test["y"].values
    y_pred = forecast.loc[forecast["ds"] == test["ds"].values[0], "yhat"].values

    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({
        "가맹점구분번호": store_id,
        "실제값": y_true[0],
        "예측값": y_pred[0],
        "MAE": mae,
        "RMSE": rmse
    })


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwlrwd6ug/p8iekleg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25266', 'data', 'file=/tmp/tmpwlrwd6ug/yofur1mt.json', 'init=/tmp/tmpwlrwd6ug/p8iekleg.json', 'output', 'file=/tmp/tmpwlrwd6ug/prophet_modelnbo4b1it/prophet_model-20251111021947.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
02:19:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:19:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 9.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwlrwd6ug/3_92pscg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwlrwd6ug/kk150z8p.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 

In [ ]:
# ⑥ 결과 DataFrame으로 정리

df_result = pd.DataFrame(results).sort_values("RMSE")
print("\n✅ 예측 결과 요약:")
print(df_result.head())


✅ 예측 결과 요약:
         가맹점구분번호  실제값  예측값  MAE  RMSE
1518  6A4747F757  0.0  0.0  0.0   0.0
1468  668C9F56AA  0.0  0.0  0.0   0.0
3327  F0228002E3  0.0  0.0  0.0   0.0
295   150889572A  0.0  0.0  0.0   0.0
3337  F0CC0C4D8F  0.0  0.0  0.0   0.0


In [ ]:
# ⑦ 평균 성능 요약

print("\n📊 전체 평균 성능")
print(f"MAE 평균: {df_result['MAE'].mean():.2f}")
print(f"RMSE 평균: {df_result['RMSE'].mean():.2f}")



📊 전체 평균 성능
MAE 평균: 3.94
RMSE 평균: 3.94


In [ ]:
# ⑧ 결과 저장 (선택)

save_path = "/content/drive/MyDrive/멋사_데이터톤/머신러닝/result/prophet_forecast_2024_12.csv"
df_result.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"\n📁 결과 저장 완료: {save_path}")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/멋사_데이터톤/머신러닝/result'